In [3]:
import numpy as np
import pandas as pd

In [4]:
import nltk
from nltk import sent_tokenize,word_tokenize 
import os

In [5]:
def get_nrc_data():
    nrc = "NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict

In [6]:
emotion_dict = get_nrc_data()

In [7]:
emotion_dict

{'abacus': ['trust'],
 'abandon': ['fear', 'negative', 'sadness'],
 'abandoned': ['anger', 'fear', 'negative', 'sadness'],
 'abandonment': ['anger', 'fear', 'negative', 'sadness', 'surprise'],
 'abba': ['positive'],
 'abbot': ['trust'],
 'abduction': ['fear', 'negative', 'sadness', 'surprise'],
 'aberrant': ['negative'],
 'aberration': ['disgust', 'negative'],
 'abhor': ['anger', 'disgust', 'fear', 'negative'],
 'abhorrent': ['anger', 'disgust', 'fear', 'negative'],
 'ability': ['positive'],
 'abject': ['disgust', 'negative'],
 'abnormal': ['disgust', 'negative'],
 'abolish': ['anger', 'negative'],
 'abolition': ['negative'],
 'abominable': ['disgust', 'fear', 'negative'],
 'abomination': ['anger', 'disgust', 'fear', 'negative'],
 'abort': ['negative'],
 'abortion': ['disgust', 'fear', 'negative', 'sadness'],
 'abortive': ['negative', 'sadness'],
 'abovementioned': ['positive'],
 'abrasion': ['negative'],
 'abrogate': ['negative'],
 'abrupt': ['surprise'],
 'abscess': ['negative', 'sad

In [26]:
def emotion_analyzer(text,emotion_dict=emotion_dict):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return emotion_count

In [27]:
directory = '../CFPB'

In [28]:
# for filename in os.listdir(directory):
#     if filename.endswith(".txt") or filename.endswith(".png"):
#         print(os.path.join(directory, filename))
#     else:
#         continue

In [29]:
res_dict = {}
evaluate_cate = ['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'negative',
 'positive',
 'sadness',
 'surprise',
 'trust']

for entry in os.scandir(directory):
    if (entry.path.endswith('.DS_Store')):
        continue
            
    print(entry.path)
    if not entry.path.endswith('.DS_Store'):
        for subentry in os.scandir(entry.path):
            if (subentry.path.endswith("Links.csv") or 
                subentry.path.endswith("Links.txt") or 
                subentry.path.endswith('.DS_Store')):
                continue
            
            if subentry.path.endswith('Rule'):
                docket_id = subentry.path.split('/')[2]
                file_type = subentry.path.split('/')[3]

                if docket_id not in res_dict:
                    res_dict[docket_id] = {}
                if file_type not in res_dict[docket_id]:
                    res_dict[docket_id][file_type] = {}

                if evaluate_cate[0] not in res_dict[docket_id][file_type] and file_type == 'Rule':
                    for i in range(len(evaluate_cate)):
                        res_dict[docket_id][file_type][evaluate_cate[i]] = []

                for filename in os.listdir(subentry):
                    if filename.endswith(".txt") and not filename.endswith("Links.txt"):
                        path_rule = os.path.join(subentry, filename)
                        f = open(path_rule, "r", encoding="ISO-8859-1")
                        text = f.read()
                        emotion_dict = emotion_analyzer(text)
                        for e in evaluate_cate:
                            res_dict[docket_id][file_type][e].append(emotion_dict[e])
                    else:
                        continue
                        
            if subentry.path.endswith('Proposed Rule'):
                    
                for sub in os.scandir(subentry.path):
                    if (sub.path.endswith("Links.csv") or 
                        sub.path.endswith("Links.txt") or 
                        sub.path.endswith('.DS_Store')):
                        continue
                    docket_id = sub.path.split('/')[2]
                    file_type = sub.path.split('/')[3]
                    pro_rule_id = sub.path.split('/')[4]

                    if docket_id not in res_dict:
                        res_dict[docket_id] = {}
                    if file_type not in res_dict[docket_id]:
                        res_dict[docket_id][file_type] = {}
                    if pro_rule_id not in res_dict[docket_id][file_type]:
                        res_dict[docket_id][file_type][pro_rule_id] = {}
                        
                    if evaluate_cate[0] not in res_dict[docket_id][file_type][pro_rule_id]:
                        for i in range(len(evaluate_cate)):
                            res_dict[docket_id][file_type][pro_rule_id][evaluate_cate[i]] = []
                            

                    for pro_rule_entry in os.scandir(sub.path):
#                         if pro_rule_entry.path == './CFPB/CFPB-2011-0023/Proposed Rule/CFPB-2011-0023-0001_content.pdf':
#                             continue
                        if (pro_rule_entry.path.endswith("Links.csv") or 
                        pro_rule_entry.path.endswith("Links.txt") or 
                        pro_rule_entry.path.endswith('.DS_Store')):
                            continue
                        
                        if pro_rule_entry.path.endswith('txt'):
                        
                            path = pro_rule_entry.path
                            f = open(path, "r", encoding="ISO-8859-1")
                            text = f.read()
                            emotion_dict = emotion_analyzer(text)
                            for e in evaluate_cate:
                                res_dict[docket_id][file_type][pro_rule_id][e].append(emotion_dict[e])
                            
                        
                        if pro_rule_entry.path.endswith('comment'):
                            if 'comment' not in res_dict[docket_id][file_type]:
                                res_dict[docket_id][file_type][pro_rule_id]['comment'] = {}
                                
                            if evaluate_cate[0] not in res_dict[docket_id][file_type][pro_rule_id]['comment']:
                                for i in range(len(evaluate_cate)):
                                    res_dict[docket_id][file_type][pro_rule_id]['comment'][evaluate_cate[i]] = []

                                
                            for filename_path in os.scandir(pro_rule_entry.path):
                                filename = filename_path.path
                                if filename.endswith(".txt") and not filename.endswith("Links.txt"):
                                    f = open(filename, "r", encoding="ISO-8859-1")
                                    text = f.read()
                                    emotion_dict = emotion_analyzer(text)
                                    for e in evaluate_cate:
                                        res_dict[docket_id][file_type][pro_rule_id]['comment'][e].append(emotion_dict[e])
                    else:
                        continue
                            
#                     for filename in os.listdir(last_sub)
#                     print(last_sub.path)
                    
                    

#                 if evaluate_cate[0] not in res_dict[docket_id][file_type]:
#                     for i in range(len(evaluate_cate)):
#                         res_dict[docket_id][file_type][evaluate_cate[i]] = []
                        


#             res_dict[docket_id][file_type].append()

../CFPB/CFPB-2012-0039
../CFPB/CFPB-2012-0037
../CFPB/CFPB-2012-0031
../CFPB/CFPB-2013-0009
../CFPB/CFPB-2014-0024
../CFPB/OCC-2012-0013
../CFPB/CFPB-2017-0021
../CFPB/CFPB-2014-0031
../CFPB/CFPB-2014-0009
../CFPB/CFPB-2013-0013
../CFPB/HUD-2013-0093
../CFPB/RHS-18-SFH-0020
../CFPB/CFPB-2012-0022
../CFPB/CFPB-2015-0006
../CFPB/CFPB-2012-0046
../CFPB/CFPB-2012-0015
../CFPB/OCC-2013-0009
../CFPB/CFPB-2012-0023
../CFPB/CFPB-2016-0038
../CFPB/CFPB-2012-0005
../CFPB/CFPB-2011-0023
../CFPB/CFPB-2012-0033
../CFPB/CFPB-2012-0034
../CFPB/OCC-2014-0002
../CFPB/CFPB-2012-0050
../CFPB/CFPB-2012-0032
../CFPB/CFPB-2015-0029
../CFPB/CFPB-2017-0008
../CFPB/CFPB-2017-0030
../CFPB/VA-2015-VBA-0004
../CFPB/CFPB-2017-0009
../CFPB/CFPB-2014-0010
../CFPB/CFPB-2013-0005
../CFPB/CFPB-2013-0002
../CFPB/CFPB-2014-0028
../CFPB/CFPB-2013-0020
../CFPB/CFPB-2013-0018
../CFPB/CFPB-2014-0003
../CFPB/OCC-2015-0021
../CFPB/CFPB-2017-0015
../CFPB/CFPB-2013-0010
../CFPB/CFPB-2014-0033
../CFPB/CFPB-2012-0021
../CFPB/CFPB-

In [31]:
res_dict_cfpb = res_dict.copy()

In [34]:
# res_dict_cfpb

In [33]:
import json

json = json.dumps(res_dict_OCC)
f = open("emotion_dict_CFPB.json","w")
f.write(json)
f.close()



In [155]:
# f = open("res_dict_OCC.json","r")